## 规划

    - mongos
     * 172.23.121.3:27027
    - config
     * 172.23.121.3:27026
     * 172.23.121.4:27026
     * 172.23.121.5:27026
    - master
     * 172.23.121.5:27025
    - slaver
     * 172.23.121.4:27025
    - arbiter
     * 172.23.121.3:27025

## 配置

#### master.conf 

    dbpath=/home/admin/var/lib/mongodb/master
    logpath=/home/admin/var/log/master.log
    pidfilepath=/home/admin/var/run/master.pid
    directoryperdb=true
    logappend=true
    replSet=SAF
    bind_ip=172.23.121.5
    port=27025
    oplogSize=10000
    fork=true
    noprealloc=true

#### slaver.conf

    dbpath=/home/admin/var/lib/mongodb/slaver
    logpath=/home/admin/var/log/slaver.log
    pidfilepath=/home/admin/var/run/slaver.pid
    directoryperdb=true
    logappend=true
    replSet=SAF
    bind_ip=172.23.121.4
    port=27025
    oplogSize=10000
    fork=true
    noprealloc=true

#### arbiter.conf

    dbpath=/home/admin/var/lib/mongodb/arbiter
    logpath=/home/admin/var/log/arbiter.log
    pidfilepath=/home/admin/var/run/arbiter.pid
    directoryperdb=true
    logappend=true
    replSet=SAF
    bind_ip=172.23.121.3
    port=27025
    oplogSize=10000
    fork=true
    noprealloc=true

#### config.conf

    dbpath=/home/admin/var/lib/mongodb/config
    logpath=/home/admin/var/log/config.log
    pidfilepath=/home/admin/var/run/config.pid
    configsvr=true
    directoryperdb=true
    logappend=true
    bind_ip=172.23.121.4
    port=27026
    oplogSize=10000
    fork=true

#### mongos.conf

    logpath=/home/admin/var/log/mongos.log
    pidfilepath=/home/admin/var/run/mongos.pid
    configdb=172.23.121.3:27026,172.23.121.4:27026,172.23.121.5:27026
    logappend=true
    bind_ip=172.23.121.3
    port=27027
    fork=true

## 启动副本集

#### 1.在master，slaver，arbiter对应的机器上以下面命令启动mongod
    mongod -f master.conf
    mongod -f slaver.conf
    mongod -f arbiter.conf

#### 2.登陆到其中任意一台机器的mongo，运行以下命令：
```javascript
use admin

config={ "_id" : "SAF", "members" : [ { "_id" : 2, "host" : "172.23.121.5:27025", "priority" : 2 }, { "_id" : 1, "host" : "172.23.121.4:27025", "priority" : 1 }, { "_id" : 0, "host" : "172.23.121.3:27025", "arbiterOnly" : true } ]  }

rs.initiate(config)
```

    说明：config可以是任意合法的变量名，但最好不要是mongodb的关键字。最外层的_id表示replica set的名字，members里包含的是所有节点的地址以及优先级。优先级最高的即成为master节点，即这里的172.23.121.5:27025。特别注意的是，对于arbiter节点，需要有个特别的配置arbiterOnly:true。这个千万不能少了，不然主备模式就不能生效。

#### 3.检验配置是否生效
    使用rs.status()检查配置是否生效，如生效应该显示信息如下：
```javascript
rs.status()
{
	"set" : "SAF",
	"date" : ISODate("2014-12-05T08:06:22Z"),
	"myState" : 7,
	"members" : [
		{
			"_id" : 0,
			"name" : "172.23.121.3:27025",
			"health" : 1,
			"state" : 7,
			"stateStr" : "ARBITER",
			"uptime" : 7007657,
			"self" : true
		},
		{
			"_id" : 1,
			"name" : "172.23.121.4:27025",
			"health" : 1,
			"state" : 2,
			"stateStr" : "SECONDARY",
			"uptime" : 7007649,
			"optime" : Timestamp(1417766780, 36),
			"optimeDate" : ISODate("2014-12-05T08:06:20Z"),
			"lastHeartbeat" : ISODate("2014-12-05T08:06:21Z"),
			"lastHeartbeatRecv" : ISODate("2014-12-05T08:06:22Z"),
			"pingMs" : 0,
			"syncingTo" : "172.23.121.5:27025"
		},
		{
			"_id" : 2,
			"name" : "172.23.121.5:27025",
			"health" : 1,
			"state" : 1,
			"stateStr" : "PRIMARY",
			"uptime" : 7007649,
			"optime" : Timestamp(1417766739, 7),
			"optimeDate" : ISODate("2014-12-05T08:05:39Z"),
			"lastHeartbeat" : ISODate("2014-12-05T08:06:20Z"),
			"lastHeartbeatRecv" : ISODate("2014-12-05T08:06:21Z"),
			"pingMs" : 0
		}
	],
	"ok" : 1
}
```

## 启动mongos

#### 1. 启动mongos和config server
    mongod -f config.conf (在三台config server上启动)
    mongos -f mongos.conf
#### 2. 配置mongos
```javascript
$ mongo 172.23.121.3:27027   #这里必须连接路由节点

> sh.addShard("SAF/172.23.121.5:27025")      # SAF表示replica set的名字,当把主节点添加到shard以后，mongos会自动找到set里的主，备，决策节点
> db.runCommand({enableSharding:"Storm"})    # "Strom" 是数据库名
> db.runCommand({shardCollection: "Storm.all",key:{"__avpSessionId":1}}) # 可选
```
    说明：第一个命令很容易理解，第二个命令是对需要进行Sharding的数据库进行配置，第三个命令是对需要进行Sharding的Collection进行配置，这里的"all"即为Collection的名字。另外还有个key，这个是比较关键的东西，对于查询效率会有很大的影响，具体可以查看Shard Key Overview
    
#### 3. 检验配置是否生效

```javascript
$ mongo 172.23.121.3:27027

> sh.status()
--- Sharding Status --- 
  sharding version: {
	"_id" : 1,
	"version" : 3,
	"minCompatibleVersion" : 3,
	"currentVersion" : 4,
	"clusterId" : ObjectId("534cca08d22d6b3631f701ad")
}
  shards:
	{  "_id" : "SAF",  "host" : "SAF/172.23.121.4:27025,172.23.121.5:27025" }
  databases:
	{  "_id" : "admin",  "partitioned" : false,  "primary" : "config" }
	{  "_id" : "test",  "partitioned" : false,  "primary" : "SAF" }
	{  "_id" : "Storm",  "partitioned" : false,  "primary" : "SAF" }
	{  "_id" : "dnsStorm",  "partitioned" : false,  "primary" : "SAF" }
```

    到这里Sharding也已经搭建完成了，以上只是最简单的搭建方式，其中某些配置仍然使用的是默认配置。如果设置不当，会导致效率异常低下，所以建议大家多看看官方文档再进行默认配置的修改。

## 创建索引
```jsp
> db.all.ensureIndex({sec:-1});
> db.idc.ensureIndex({sec:-1, idc:1});
> db.isp.ensureIndex({sec:-1, isp:1});
> db.reg.ensureIndex({sec:-1, reg:1});
> db.reg.idc.ensureIndex({sec:-1, reg:1, idc:1});
> db.reg.isp.ensureIndex({sec:-1, reg:1, isp:1});
```